In [48]:
import torch
from safetensors.torch import load_file, save_file
from pprint import pprint

In [52]:
original_ckpt_path = "/home/thomwolf/Documents/Github/ACT/checkpoints/blue_red_sort/policy_last.ckpt"
converted_ckpt_path = "/home/thomwolf/Documents/Github/ACT/checkpoints/blue_red_sort/model.safetensors"

comparison_main_path = "/home/thomwolf/Documents/Github/lerobot/examples/real_robot_example/outputs/train/blue_red_debug_no_masking/checkpoints/last/pretrained_model/"
comparison_safetensor_path = comparison_main_path + "model.safetensors"
comparison_config_json_path = comparison_main_path + "config.json"
comparison_config_yaml_path = comparison_main_path + "config.yaml"

In [28]:
a = torch.load(original_ckpt_path)

In [6]:
b = load_file(comparison_safetensor_path)

In [17]:
dest = list(b.keys())
pprint(dest)

['model.action_head.bias',
 'model.action_head.weight',
 'model.backbone.bn1.bias',
 'model.backbone.bn1.running_mean',
 'model.backbone.bn1.running_var',
 'model.backbone.bn1.weight',
 'model.backbone.conv1.weight',
 'model.backbone.layer1.0.bn1.bias',
 'model.backbone.layer1.0.bn1.running_mean',
 'model.backbone.layer1.0.bn1.running_var',
 'model.backbone.layer1.0.bn1.weight',
 'model.backbone.layer1.0.bn2.bias',
 'model.backbone.layer1.0.bn2.running_mean',
 'model.backbone.layer1.0.bn2.running_var',
 'model.backbone.layer1.0.bn2.weight',
 'model.backbone.layer1.0.conv1.weight',
 'model.backbone.layer1.0.conv2.weight',
 'model.backbone.layer1.1.bn1.bias',
 'model.backbone.layer1.1.bn1.running_mean',
 'model.backbone.layer1.1.bn1.running_var',
 'model.backbone.layer1.1.bn1.weight',
 'model.backbone.layer1.1.bn2.bias',
 'model.backbone.layer1.1.bn2.running_mean',
 'model.backbone.layer1.1.bn2.running_var',
 'model.backbone.layer1.1.bn2.weight',
 'model.backbone.layer1.1.conv1.weight',


In [29]:
orig = list(a.keys())
pprint(orig)

['model.pos_table',
 'model.transformer.encoder.layers.0.self_attn.in_proj_weight',
 'model.transformer.encoder.layers.0.self_attn.in_proj_bias',
 'model.transformer.encoder.layers.0.self_attn.out_proj.weight',
 'model.transformer.encoder.layers.0.self_attn.out_proj.bias',
 'model.transformer.encoder.layers.0.linear1.weight',
 'model.transformer.encoder.layers.0.linear1.bias',
 'model.transformer.encoder.layers.0.linear2.weight',
 'model.transformer.encoder.layers.0.linear2.bias',
 'model.transformer.encoder.layers.0.norm1.weight',
 'model.transformer.encoder.layers.0.norm1.bias',
 'model.transformer.encoder.layers.0.norm2.weight',
 'model.transformer.encoder.layers.0.norm2.bias',
 'model.transformer.encoder.layers.1.self_attn.in_proj_weight',
 'model.transformer.encoder.layers.1.self_attn.in_proj_bias',
 'model.transformer.encoder.layers.1.self_attn.out_proj.weight',
 'model.transformer.encoder.layers.1.self_attn.out_proj.bias',
 'model.transformer.encoder.layers.1.linear1.weight',
 '

In [45]:
a = torch.load(original_ckpt_path)

to_remove_startswith = ['model.transformer.decoder.layers.1.',
             'model.transformer.decoder.layers.2.',
             'model.transformer.decoder.layers.3.',
             'model.transformer.decoder.layers.4.',
             'model.transformer.decoder.layers.5.',
             'model.transformer.decoder.layers.6.',
             'model.transformer.decoder.norm.',
             'model.is_pad_head']

to_remove_in = ['num_batches_tracked',]

conv = {}

keys = list(a.keys())
for k in keys:
    if any(k.startswith(tr) for tr in to_remove_startswith):
        a.pop(k)
        continue
    if any(tr in k for tr in to_remove_in):
        a.pop(k)
        continue
    if k.startswith('model.transformer.encoder.layers.'):
        conv[k.replace('transformer.', '')] = a.pop(k)
    if k.startswith('model.transformer.decoder.layers.0.'):
        conv[k.replace('transformer.', '')] = a.pop(k)
    if k.startswith('model.encoder.layers.'):
        conv[k.replace('encoder.', 'vae_encoder.')] = a.pop(k)
    if k.startswith('model.action_head.'):
        conv[k] = a.pop(k)
    if k.startswith('model.pos_table'):
        conv[k.replace('pos_table', 'vae_encoder_pos_enc')] = a.pop(k)
    if k.startswith('model.query_embed.'):
        conv[k.replace('query_embed', 'decoder_pos_embed')] = a.pop(k)
    if k.startswith('model.input_proj.'):
        conv[k.replace('input_proj.', 'encoder_img_feat_input_proj.')] = a.pop(k)
    if k.startswith('model.input_proj_robot_state.'):
        conv[k.replace('input_proj_robot_state.', 'encoder_robot_state_input_proj.')] = a.pop(k)
    if k.startswith('model.backbones.0.0.body.'):
        conv[k.replace('backbones.0.0.body', 'backbone')] = a.pop(k)
    if k.startswith('model.cls_embed.'):
        conv[k.replace('cls_embed', 'vae_encoder_cls_embed')] = a.pop(k)
    if k.startswith('model.encoder_action_proj.'):
        conv[k.replace('encoder_action_proj', 'vae_encoder_action_input_proj')] = a.pop(k)
    if k.startswith('model.encoder_joint_proj.'):
        conv[k.replace('encoder_joint_proj', 'vae_encoder_robot_state_input_proj')] = a.pop(k)
    if k.startswith('model.latent_proj.'):
        conv[k.replace('latent_proj', 'vae_encoder_latent_output_proj')] = a.pop(k)
    if k.startswith('model.latent_out_proj.'):
        conv[k.replace('latent_out_proj', 'encoder_latent_input_proj')] = a.pop(k)
    if k.startswith('model.additional_pos_embed.'):
        conv[k.replace('additional_pos_embed', 'encoder_robot_and_latent_pos_embed')] = a.pop(k)

In [46]:
a

OrderedDict()

In [47]:
for k, v in conv.items():
    assert b[k].shape == v.shape
    b[k] = v

In [53]:
save_file(b, converted_ckpt_path)

In [54]:
# Now also copy the config files
import shutil
shutil.copy(comparison_config_json_path, converted_ckpt_path.replace('model.safetensors', 'config.json'))
shutil.copy(comparison_config_yaml_path, converted_ckpt_path.replace('model.safetensors', 'config.yaml'))

'/home/thomwolf/Documents/Github/ACT/checkpoints/blue_red_sort/config.yaml'